# Traitement DB prpriétaires #

# Import libraries #

In [1]:
import pandas as pd
import sqlite3

# DB connexion #

In [2]:
db_file = '../data/APMK.sqlite'
con = sqlite3.connect(db_file)

# Load data #

## load communes ##

In [4]:
communes = pd.read_sql_query("SELECT * from communes", con)
communes.head(3)

,no_commune,nom_commune
0,5421,Apples
1,5423,Ballens
2,5432,Montherod


## load potential land usage ##

In [7]:
land_usage = pd.read_sql_query("SELECT nature FROM types_nature WHERE potentiel = 1", con)
land_usage.head(3)

,nature
0,Jardin
1,"Champ, pré, pâturage"
2,Forêt


## load public parcelles ##

In [35]:
fields = ['no_commune', 'nom_commune', 'immeuble', 'surface', 'proprietaire', 'type_nature', 'surface_nature']
listToStr = ', '.join([str(elem) for elem in fields])
parcelles_conf = pd.read_sql_query("SELECT DISTINCT " + listToStr + " FROM APMK WHERE proprietaire LIKE 'Conf%'", con)
parcelles_conf.head(3)

,no_commune,nom_commune,immeuble,surface,proprietaire,type_nature,surface_nature
0,5423,Ballens,689,24461.0,"Confédération Suisse, Etat-major général, Div....",Forêt,24461.0
1,5432,Montherod,199,9474.0,"Confédération Suisse, Etat-major général, Div....",Forêt,9474.0
2,5432,Montherod,205,13093.0,"Confédération Suisse, Etat-major général, Div....",Forêt,13093.0


### check data ###

In [36]:
len(parcelles_conf)

261

# Find parcelles own by Confédération with potatial #

In [44]:
parcelles_conf_pot = parcelles_conf[parcelles_conf['type_nature'].isin(land_usage['nature'])]
parcelles_conf_pot.head()

,no_commune,nom_commune,immeuble,surface,proprietaire,type_nature,surface_nature
0,5423,Ballens,689,24461.0,"Confédération Suisse, Etat-major général, Div....",Forêt,24461.0
1,5432,Montherod,199,9474.0,"Confédération Suisse, Etat-major général, Div....",Forêt,9474.0
2,5432,Montherod,205,13093.0,"Confédération Suisse, Etat-major général, Div....",Forêt,13093.0
3,5432,Montherod,301,38374.0,"Confédération Suisse, Etat-major général, Div....",Forêt,38374.0
4,5432,Montherod,302,3906.0,"Confédération Suisse, Etat-major général, Div....",Forêt,3906.0


In [45]:
len(parcelles_conf_pot)

142

# Import data of communes of Vaud #

In [49]:
file_communes_vd = '../data/communes_vd.csv'
communes_vd_data = pd.read_csv(file_communes_vd)
# cleanup
communes_vd_data = communes_vd_data.drop(['GEOMETRY'], axis=1)

In [51]:
communes_vd_data = communes_vd_data[['ogc_fid', 'no_com_can', 'no_com_fed', 'nom_min', 'perimetre']]

In [53]:
communes_vd_data.head(3)

,ogc_fid,no_com_can,no_com_fed,nom_min,perimetre
0,1,102,5535,Saint-Barthélemy (VD),NaN
1,2,336,5861,Rolle,NaN
2,3,381,5932,Ursins,NaN


## Add no commune Vaud ##

In [62]:
communes_no = communes_vd_data[['no_com_fed', 'no_com_can', ]]
communes_no = communes_no.rename(columns={"no_com_fed": "no_commune"})

In [65]:
parcelles_conf_pot = pd.merge(parcelles_conf_pot, communes_no, how='left', on='no_commune')

### check data ###

In [66]:
parcelles_conf_pot.head(3)

,no_commune,nom_commune,immeuble,surface,proprietaire,type_nature,surface_nature,no_com_can
0,5423,Ballens,689,24461.0,"Confédération Suisse, Etat-major général, Div....",Forêt,24461.0,18
1,5432,Montherod,199,9474.0,"Confédération Suisse, Etat-major général, Div....",Forêt,9474.0,27
2,5432,Montherod,205,13093.0,"Confédération Suisse, Etat-major général, Div....",Forêt,13093.0,27


In [70]:
parcelles_conf_pot.no_com_can.unique()

array([ 18,  27,  30,  46,  47,  51,  54,  65,  72,  76,  96, 127, 130,
       132, 133, 136, 138, 157, 160, 168, 170, 171, 175, 176, 181, 184,
       185, 351])

## build IDEX2000 value ##

In [73]:
parcelles_conf_pot['IDEX2000'] = parcelles_conf_pot['no_com_can'].map('{:0>3}'.format) + parcelles_conf_pot['immeuble'].map('{:0>6}'.format)

In [74]:
parcelles_conf_pot

,no_commune,nom_commune,immeuble,surface,proprietaire,type_nature,surface_nature,no_com_can,IDEX2000
0,5423,Ballens,689,24461.0,"Confédération Suisse, Etat-major général, Div....",Forêt,24461.0,18,018000689
1,5432,Montherod,199,9474.0,"Confédération Suisse, Etat-major général, Div....",Forêt,9474.0,27,027000199
2,5432,Montherod,205,13093.0,"Confédération Suisse, Etat-major général, Div....",Forêt,13093.0,27,027000205
3,5432,Montherod,301,38374.0,"Confédération Suisse, Etat-major général, Div....",Forêt,38374.0,27,027000301
4,5432,Montherod,302,3906.0,"Confédération Suisse, Etat-major général, Div....",Forêt,3906.0,27,027000302
...,...,...,...,...,...,...,...,...,...
137,5652,Villars-sous-Yens,513,540.0,Confédération Suisse Département Militaire Féd...,Place-jardin,423.0,185,185000513
138,5902,Belmont-sur-Yverdon,651,40938.0,Confédération Suisse Office fédéral des routes...,"Champ, pré, pâturage",13612.0,351,351000651
139,5902,Belmont-sur-Yverdon,651,40938.0,Confédération Suisse Office fédéral des routes...,Forêt,4286.0,351,351000651
140,5902,Belmont-sur-Yverdon,652,51949.0,Confédération Suisse Office fédéral des routes...,"Champ, pré, pâturage",29114.0,351,351000652


# Export to csv #

In [75]:
parcelles_conf_pot.to_csv('../data/parcelles_conf_pot.csv', index=False)